In [1]:
import numpy as np
import pandas as pd
from graphviz import Digraph
from scipy.io import arff
import time
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import queue
import math

In [4]:
data = arff.loadarff('D:/Projects/datasets/monks-problem.arff')
data = pd.DataFrame(data[0])
data_labels = data['class']
data = data.drop(columns = ['class'])
data_labels[np.array(list(map(int, data_labels))) == min(list(map(int,data_labels.unique())))] = '0'
data_labels[np.array(list(map(int, data_labels))) == max(list(map(int,data_labels.unique())))] = '1'

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Projects/datasets/monks-problem.arff'

In [5]:
for column in data.columns:
    data[column] = list(map(int,data[column]))

NameError: name 'data' is not defined

In [ ]:
model = XGBClassifier()
X_train, X_test, y_train, y_test = train_test_split(data, data_labels, test_size = 0.25)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
acc_score(predicted, y_test)

In [6]:
def acc_score(y_predicted, y_labels):
    return (y_predicted == y_labels).sum()/len(y_labels)

In [7]:
class node:
    def __init__(self, node_name):
        self.node_name = node_name
        self.child_nodes = []
        self.T = 0
        self.score_sum = 0
        
    def add_child_node(self, node_name):
        new_node = node(node_name)
        self.child_nodes.append(new_node)
        
    def add_child_nodes(self, node_names):
        for name in node_names:
            self.add_child_node(name)
      
    def add_score(self, score):
        self.score_sum += score
        self.T += 1
    
    def get_score(self):
        return float('Inf') if self.T == 0 else self.score_sum/self.T

In [8]:
class MCTS_tree:
    def __init__(self, score_metric, max_iter = None, max_time = None, verbosity = 1):
        self._root = node("root")
        self._score_metric = score_metric
        self._verbosity = verbosity
        self._max_iter = max_iter
        self._max_time = max_time
        self._model = XGBClassifier()
        self._best_score = 0
        self._current_best_variables = []
        
    def _node_score(self, node, parent_node):
        return node.get_score() + 0 if node.T == 0 else math.sqrt(2*math.log(parent_node.T)/node.T)
    
    def _calculate_score(self, feature_names):
        print('Calcualting for ' + ' '.join(feature_names))
        X_train, X_test, y_train, y_test = train_test_split(self._data.loc[:, feature_names], self._labels, test_size = 0.25)
        self._model.fit(X_train, y_train)
        predicted = self._model.predict(X_test)
        
        tmp_score = self._score_metric(predicted, y_test)
        
        if(tmp_score > self._best_score):
            self._best_score = tmp_score
            self._current_best_variables = feature_names
        
        return tmp_score
    
    def _update_nodes(self, nodes, score):
        self._root.add_score(score)
        
        for node in nodes:
            node.add_score(score)
    
    def _add_child_nodes(self, node, selected_nodes):
        nodes_to_add = []
        
        if(len(selected_nodes) == 0):
            node.add_child_nodes(self._data.columns)
        else:
            for column in self._data:
                is_used = False
                for n in selected_nodes:
                    if(n.node_name == column):
                        is_used = True
                        break
                if(not is_used):
                    nodes_to_add.append(column)
            print('Adding child nodes: ' + ' '.join(nodes_to_add))
            node.add_child_nodes(nodes_to_add)
    
    def _single_iteration(self):
        current_node = self._root
        selected_nodes = []
        stop_val = False
        
        while(not stop_val):
            best_score = 0
            selected_node = None
            
            if(current_node.T == 0):
                print('First if: ' + current_node.node_name)
                if(len(current_node.child_nodes) != 0):
                    print("Inside first if appending first node")
                    selected_nodes.append(current_node.child_nodes[0])
                stop_val = True
            else:
                print('Else')
                if(len(current_node.child_nodes) == 0):
                    print('Adding child nodes for ' + current_node.node_name)
                    self._add_child_nodes(current_node, selected_nodes)
                    if(len(current_node.child_nodes) != 0):
                        print('Adding first child: ' + current_node.child_nodes[0].node_name)
                        selected_nodes.append(current_node.child_nodes[0])
                    stop_val = True
                else:
                    print('Going recursive')
                    for node in current_node.child_nodes:
                        current_score = self._node_score(node, current_node)
                        if(current_score > best_score):
                            best_score = current_score
                            selected_node = node
                    selected_nodes.append(selected_node)
                    current_node = selected_node
                    print('Selected node: ' + selected_node.node_name)
         
        feature_names = []
        for node in selected_nodes:
            feature_names.append(node.node_name)   
        score = self._calculate_score(feature_names)    
        
        if(self._verbosity == 1):
            print('Current best score: ' + str(score))
        
        self._update_nodes(selected_nodes, score)
    
    def _are_calculations_over(self):
        if(self._max_time != None):
            return time.time() - self._start >= self._max_time
        else:
            return self._iterations >= self._max_iter
            
    def _start_fitting(self):
        self._start = time.time()
        self._iterations = 0
        
        self._add_child_nodes(self._root,[])
        
        while(not self._are_calculations_over()):
            print("------------------------")
            self._single_iteration()
            
            self._iterations += 1
        
    
    def fit(self, data, labels):
        self._data = data
        self._labels = labels
        
        self._start_fitting()

In [9]:
tree = MCTS_tree(max_iter = 1000, score_metric=acc_score)
tree.fit(data, data_labels)

NameError: name 'data' is not defined

In [88]:
print(tree._current_best_variables)
print(tree._best_score)

['attr4', 'attr2', 'attr6', 'attr3', 'attr1']
0.8079470198675497
